In [1]:
# Py3_Jupyter_Nb_Pandas(股票同比分析)_GF_2024-01-16.ipynb
# Create By GF 2024-01-16 12:09

In [2]:
import zhdate # -> ZhDate Version 0.1
import datetime
import pandas as pd # -> Pandas Version 1.4.1

In [3]:
def MapFunc_DateTime_Judgement_Leap_Year(DateTime:object) -> str:
    
    """
    [Require] import datetime
    
    [Explain] * Divide the number of Gregorian years by 4, and if you get an integer, then this year is a leap year.
                公历年数除以 4, 如果得到的是整数, 那这一年就是闰年。
              
              * If the number of years in the Gregorian calendar is a multiple of 100 and then divided by 400, the result is an integer, which is also a leap year.
                公历年数是 100 的整数倍, 再除以 400, 得到的是整数, 那这一年也是闰年。
              
              * For example, dividing 2020 by 4 can result in an integer division, which means 2020 is a leap year.
                例如: 2020年 除以 4 可以整除, 即 2020 年为闰年。
    """
    Year:int = 0
    
    # ----------------------------------------------
    if (type(DateTime) is datetime.datetime): Year = DateTime.year
    # ..............................................
    if (type(DateTime) is str): Year = int(DateTime[0:4])
    
    # ----------------------------------------------
    # Non whole hundred year judgment method:
    if   (Year % 4   == 0):
        
        return str("闰年") # -> 闰年 (Leap Year)
    
    # whole hundred year judgment method:
    elif (Year % 100 == 0) and (Year % 400 == 0):
        
        return str("闰年") # -> 闰年 (Leap Year)
    
    else:
        
        return str("平年") # -> 闰年 (Common Year)
    # ##############################################
    # End of Function.

In [4]:
StkPDF = pd.read_csv("./Datas/Stock.csv")

In [5]:
# 转换类型: 初次读取后转换。
# ##################################################
StkPDF["Date"] = StkPDF["Date"].astype("datetime64[ns]")

# ##################################################
StkPDF

,Date,Code,Open,High,Low,Close,Pre_Close,Change,Chg_Pct,Turnover,Volume,Amount,Circle,Total
0,2015-12-31,'000422,7.93,7.95,7.76,7.77,7.93,-0.16,-0.020177,0.015498,13915200,109318000,6976420000,6976420000
1,2015-12-30,'000422,7.86,7.93,7.75,7.93,7.84,0.09,0.011480,0.018662,16755900,131567000,7120080000,7120080000
2,2015-12-29,'000422,7.72,7.85,7.69,7.84,7.71,0.13,0.016861,0.015886,14263800,110789000,7039280000,7039280000
3,2015-12-28,'000422,8.03,8.08,7.70,7.71,8.03,-0.32,-0.039851,0.030821,27672800,218869000,6922550000,6922550000
4,2015-12-25,'000422,8.03,8.05,7.93,8.03,7.99,0.04,0.005006,0.021132,18974000,151673000,7209870000,7209870000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2647,2005-02-04,'000422,6.75,7.05,6.71,7.00,6.75,0.25,0.037037,0.017178,2430800,16818500,1725750000,990538000
2648,2005-02-03,'000422,7.00,7.15,6.73,6.75,6.90,-0.15,-0.021739,0.028556,4040880,28086200,1664110000,955162000
2649,2005-02-02,'000422,6.42,6.99,6.42,6.90,6.42,0.48,0.074766,0.032928,4659550,31345900,1701090000,976388000
2650,2005-02-01,'000422,6.78,6.89,6.30,6.42,6.81,-0.39,-0.057269,0.027348,3869880,25333700,1582760000,908465000


In [6]:
# 聚合数据: 按公历年月聚合涨跌额。
# ##################################################
SourceDataPDF = StkPDF

# 提取数据: 提取公历年月。
# --------------------------------------------------
TEMPPDF = SourceDataPDF
# ..................................................
TEMPPDF["Year_Month"] = TEMPPDF["Date"].apply(lambda X: "%02d-%02d" % (X.year, X.month))
# ..................................................
StkYearMonthPDF = TEMPPDF

# 聚合数据: 按公历年月聚合涨跌额。
# --------------------------------------------------
TEMPPDF = StkYearMonthPDF[["Year_Month", "Change"]]
# ..................................................
TEMPPDF = TEMPPDF.groupby("Year_Month", as_index=False).sum("Change")
# ..................................................
StkYearMonthPDF = TEMPPDF

# ##################################################
StkYearMonthPDF.sort_values("Year_Month")

,Year_Month,Change
0,2005-01,-0.13
1,2005-02,0.52
2,2005-03,-0.29
3,2005-04,0.62
4,2005-05,-1.23
...,...,...
127,2015-08,-0.88
128,2015-09,-0.55
129,2015-10,1.00
130,2015-11,0.07


In [7]:
# 计算数据: 按公历年月提取同比(YOY)数据。
# ##################################################
SourceDataPDF = StkYearMonthPDF.sort_values("Year_Month", ascending=False)

# 提取数据: 提取公历N年前数据。
# --------------------------------------------------
TEMPPDF = SourceDataPDF
# ..................................................
TEMPPDF["Pre1_Year_Month"] = TEMPPDF["Year_Month"].shift(-12)
TEMPPDF["Pre1_Change"]     = TEMPPDF["Change"].shift(-12)
# ..................................................
TEMPPDF["Pre2_Year_Month"] = TEMPPDF["Year_Month"].shift(-24)
TEMPPDF["Pre2_Change"]     = TEMPPDF["Change"].shift(-24)
# ..................................................
TEMPPDF["Pre3_Year_Month"] = TEMPPDF["Year_Month"].shift(-36)
TEMPPDF["Pre3_Change"]     = TEMPPDF["Change"].shift(-36)
# ..................................................
StkYearMonthYoyPDF = TEMPPDF

# 提取特征: 提取公历年月同比涨跌特征。
# --------------------------------------------------
TEMPPDF = StkYearMonthYoyPDF
# ..................................................
for Idx in TEMPPDF.index:
    Pre1_Chg = TEMPPDF.loc[Idx, "Pre1_Change"]
    Pre2_Chg = TEMPPDF.loc[Idx, "Pre2_Change"]
    Pre3_Chg = TEMPPDF.loc[Idx, "Pre3_Change"]
    # ..............................................
    if Pre1_Chg >= 0:
        Pre1_Feature = str("1")
    else:
        Pre1_Feature = str("0")
    # ..............................................
    if Pre2_Chg >= 0:
        Pre2_Feature = str("1")
    else:
        Pre2_Feature = str("0")
    # ..............................................
    if Pre3_Chg >= 0:
        Pre3_Feature = str("1")
    else:
        Pre3_Feature = str("0")
    # ..............................................
    TEMPPDF.loc[Idx, "YOY_Features"] = "%s,%s,%s" % (Pre1_Feature, Pre2_Feature, Pre3_Feature)
# ..................................................
StkYearMonthYoyPDF = TEMPPDF

# ##################################################
StkYearMonthYoyPDF.head(12)

,Year_Month,Change,Pre1_Year_Month,Pre1_Change,Pre2_Year_Month,Pre2_Change,Pre3_Year_Month,Pre3_Change,YOY_Features
131,2015-12,0.44,2014-12,0.87,2013-12,-0.43,2012-12,2.08,"1,0,1"
130,2015-11,0.07,2014-11,0.49,2013-11,0.22,2012-11,-1.56,"1,1,0"
129,2015-10,1.00,2014-10,-0.14,2013-10,-0.46,2012-10,-1.55,"0,0,0"
128,2015-09,-0.55,2014-09,0.54,2013-09,-0.04,2012-09,0.38,"1,0,1"
127,2015-08,-0.88,2014-08,0.25,2013-08,0.59,2012-08,-0.54,"1,1,0"
126,2015-07,-1.53,2014-07,0.48,2013-07,-0.26,2012-07,0.05,"1,0,1"
125,2015-06,-0.27,2014-06,-0.05,2013-06,-2.56,2012-06,-2.05,"0,0,0"
124,2015-05,0.61,2014-05,0.11,2013-05,0.60,2012-05,0.54,"1,1,1"
123,2015-04,0.57,2014-04,-0.23,2013-04,-1.59,2012-04,1.70,"0,0,1"
122,2015-03,0.96,2014-03,-0.39,2013-03,-1.79,2012-03,-1.29,"0,0,0"


In [8]:
# 聚合数据: 按农历年月聚合涨跌额。
# ##################################################
SourceDataPDF = StkPDF

# 提取数据: 提取农历日期。
# --------------------------------------------------
TEMPPDF = SourceDataPDF
# ..................................................
TEMPPDF["Zh_Date"] = TEMPPDF["Date"].apply(lambda X: zhdate.ZhDate.from_datetime(X))
# ..................................................
StkZhYearMonthPDF = TEMPPDF

# 提取数据: 判断农历闰月。
# --------------------------------------------------
TEMPPDF = SourceDataPDF
# ..................................................
TEMPPDF["Leap_Month"] = TEMPPDF["Zh_Date"].map(lambda X: "闰月" if (X.leap_month == True) else "非闰月")
# ..................................................
StkZhYearMonthPDF = TEMPPDF

# 提取数据: 筛选出农历 [非闰月] 数据。
# --------------------------------------------------
TEMPPDF = SourceDataPDF
# ..................................................
TEMPPDF = TEMPPDF[TEMPPDF["Leap_Month"] == "非闰月"]
# ..................................................
StkZhYearMonthPDF = TEMPPDF

# 提取数据: 提取农历年月。
# --------------------------------------------------
TEMPPDF = SourceDataPDF
# ..................................................
TEMPPDF["Zh_Year_Month"] = TEMPPDF["Zh_Date"].apply(lambda X: "%02d-%02d" % (X.lunar_year, X.lunar_month))
# ..................................................
StkZhYearMonthPDF = TEMPPDF

# 聚合数据: 按公历年月聚合涨跌额。
# --------------------------------------------------
TEMPPDF = StkZhYearMonthPDF[["Zh_Year_Month", "Change"]]
# ..................................................
TEMPPDF = TEMPPDF.groupby("Zh_Year_Month", as_index=False).sum("Change")
# ..................................................
StkZhYearMonthPDF = TEMPPDF

# ##################################################
StkZhYearMonthPDF

,Zh_Year_Month,Change
0,2004-12,0.06
1,2005-01,0.62
2,2005-02,0.08
3,2005-03,-0.04
4,2005-04,-1.61
...,...,...
127,2015-07,-1.57
128,2015-08,0.01
129,2015-09,0.73
130,2015-10,0.06


In [9]:
# 计算数据: 按农历年月提取同比(YOY)数据。
# ##################################################
SourceDataPDF = StkZhYearMonthPDF.sort_values("Zh_Year_Month", ascending=False)

# 提取数据: 提取公历N年前数据。
# --------------------------------------------------
TEMPPDF = SourceDataPDF
# ..................................................
TEMPPDF["Pre1_Zh_Year_Month"] = TEMPPDF["Zh_Year_Month"].shift(-12)
TEMPPDF["Pre1_Change"]        = TEMPPDF["Change"].shift(-12)
# ..................................................
TEMPPDF["Pre2_Zh_Year_Month"] = TEMPPDF["Zh_Year_Month"].shift(-24)
TEMPPDF["Pre2_Change"]        = TEMPPDF["Change"].shift(-24)
# ..................................................
TEMPPDF["Pre3_Zh_Year_Month"] = TEMPPDF["Zh_Year_Month"].shift(-36)
TEMPPDF["Pre3_Change"]        = TEMPPDF["Change"].shift(-36)
# ..................................................
StkZhYearMonthYoyPDF = TEMPPDF

# 提取特征: 提取农历年月同比涨跌特征。
# --------------------------------------------------
TEMPPDF = StkZhYearMonthYoyPDF
# ..................................................
for Idx in TEMPPDF.index:
    Pre1_Chg = TEMPPDF.loc[Idx, "Pre1_Change"]
    Pre2_Chg = TEMPPDF.loc[Idx, "Pre2_Change"]
    Pre3_Chg = TEMPPDF.loc[Idx, "Pre3_Change"]
    # ..............................................
    if Pre1_Chg >= 0:
        Pre1_Feature = str("1")
    else:
        Pre1_Feature = str("0")
    # ..............................................
    if Pre2_Chg >= 0:
        Pre2_Feature = str("1")
    else:
        Pre2_Feature = str("0")
    # ..............................................
    if Pre3_Chg >= 0:
        Pre3_Feature = str("1")
    else:
        Pre3_Feature = str("0")
    # ..............................................
    TEMPPDF.loc[Idx, "Zh_YOY_Features"] = "%s,%s,%s" % (Pre1_Feature, Pre2_Feature, Pre3_Feature)
# ..................................................
StkZhYearMonthYoyPDF = TEMPPDF

# ##################################################
StkZhYearMonthYoyPDF.head(12)

,Zh_Year_Month,Change,Pre1_Zh_Year_Month,Pre1_Change,Pre2_Zh_Year_Month,Pre2_Change,Pre3_Zh_Year_Month,Pre3_Change,Zh_YOY_Features
131,2015-11,0.10,2014-11,-0.58,2013-11,-0.16,2012-11,0.74,"0,0,1"
130,2015-10,0.06,2014-10,0.97,2013-10,-0.11,2012-10,0.40,"1,0,1"
129,2015-09,0.73,2014-09,0.27,2013-09,-0.40,2012-09,-2.76,"1,0,0"
128,2015-08,0.01,2014-08,0.21,2013-08,-0.20,2012-08,-0.22,"1,0,0"
127,2015-07,-1.57,2014-07,0.50,2013-07,0.25,2012-07,0.12,"1,1,1"
126,2015-06,1.11,2014-06,0.41,2013-06,-0.09,2012-06,-0.22,"1,0,0"
125,2015-05,-4.99,2014-05,-0.18,2013-05,-1.51,2012-05,-0.61,"0,0,0"
124,2015-04,3.81,2014-04,0.24,2013-04,-0.27,2012-04,-0.08,"1,0,0"
123,2015-03,-0.23,2014-03,-0.37,2013-03,-1.66,2012-03,-0.65,"0,0,0"
122,2015-02,0.80,2014-02,-0.30,2013-02,-0.68,2012-02,0.87,"0,0,1"
